In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Dense, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import initializers

In [2]:
os.environ["KERAS_BACKEND"] = "tensorflow"

np.random.seed(10)
#random noise vector
random_dim = 100

In [4]:
def load_minst_data():
  #making tezt and training data 
  (x_train, y_train), (x_test, y_test) = mnist.load_data()
  #normalize our inputs to be in range (-1,1)
  x_train = (x_train.astype(np.float32) - 127.5)/127.5
  #make 784 colunms and 60000 rows
  x_train = x_train.reshape(60000, 784)
  return (x_train, y_train, x_test, y_test)

In [5]:
def get_optimizer():
  return Adam(lr=0.0002, beta_1=0.5)
  #beta2. The exponential decay rate for the second-moment 
  #estimates (e.g. 0.999). This value should be set close 
  #to 1.0 on problems with a sparse gradient (e.g. NLP and computer vision problems).
  #lr is learn rate is how fast is learns it is important
  #if it is too big the model will try to find the answer too quick
def get_generator(optimizer):
  generator = Sequential() #generator replaces model because they are two NN
  generator.add(Dense(256, input_dim=random_dim, kernel_initializer=initializers.RandomNormal(stddev=0.02))) #confusion
  generator.add(LeakyReLU(0.2))
  
  generator.add(Dense(512)) #when to add activations
  generator.add(LeakyReLU(0.2))

  generator.add(Dense(1024))
  generator.add(LeakyReLU(0.2))

  generator.add(Dense(784, activation="tanh"))
  generator.compile(loss="binary_crossentropy", optimizer=optimizer)
  return generator

def get_discriminator(optimizer):
  discriminator = Sequential()
  discriminator.add(Dense(1024, input_dim=784, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))

  discriminator.add(Dense(512))
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))

  discriminator.add(Dense(256))
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))

  discriminator.add(Dense(1, activation= "sigmoid",))
  discriminator.compile(loss="binary_crossentropy", optimizer=optimizer)
  return discriminator

In [6]:
def get_gan_network(discriminator, random_dim, generator, optimizer):
  #set trainable to flase cause train discrimintor & generator 
  discriminator.trainable = False
  #gan input (noise) will be 100-dimensional vectors
  gan_input = Input(shape=(random_dim,))
  #output is an image
  x = generator(gan_input)
  #get the ouput of discrimnator
  gan_output = discriminator(x)
  gan = Model(inputs=gan_input, outputs=gan_output)
  gan.compile(loss="binary_crossentropy", optimizer=optimizer)
  return gan

In [7]:
#create a wall of generated MInST images
def plot_generated_images(epoch, generator, examples=100, dim=(10, 10), figsize=(10, 10)):
  noise = np.random.normal(0, 1, size=[examples, random_dim])
  generated_images = generator.predict(noise)
  generated_images = generated_images.reshape(examples, 28, 28)

  plt.figure(figsize=figsize)
  for i in range(generated_images.shape[0]):
    plt.subplot(dim[0], dim[1], i+1)
    plt.imshow(generated_images[i], interpolation="nearest", cmap="gray_r")
    plt.axis("off")
  plt.tight_layout()
  plt.savefig("gan_generated_image_epoch_%d.png" % epoch)

In [9]:
def train(epochs=1, batch_size=128):
  x_train, y_train, x_test, y_test = load_minst_data()
  batch_count = x_train.shape[0] / batch_size
  adam = get_optimizer()
  generator = get_generator(adam)
  discriminator = get_discriminator(adam)
  gan = get_gan_network(discriminator, random_dim, generator, adam)
  #random noise and images
  for e in range(1, epochs+1):
    print("-"*15,"Epoch %d" % e, "-"*15)
    for _ in tqdm(range(int(batch_count))):
      noise = np.random.normal(0, 1, size=[batch_size, random_dim])
      image_batch = x_train[np.random.randint(0, x_train.shape[0], size=batch_size)]
      
      #generate fake images
      generated_images = generator.predict(noise)
      X = np.concatenate([image_batch, generated_images])

      #labels for fake + real
      y_dis = np.zeros(2*batch_size)
      y_dis[:batch_size] = 0.9

      discriminator.trainable = True 
      discriminator.train_on_batch(X, y_dis)

      noise = np.random.normal(0, 1, size=[batch_size, random_dim])
      y_gen = np.ones(batch_size)
      discriminator.trainable = False
      gan.train_on_batch(noise, y_gen)

    if e == 1 or e % 20 == 0:
      plot_generated_images(e, generator)
      
#if __name__ == "__main__":
#train(400, 128)


In [ ]:
train(400, 128)

11493376/11490434 [==============================] - 0s 0us/step


  0%|          | 0/468 [00:00<?, ?it/s]

--------------- Epoch 1 ---------------


  1%|          | 3/468 [00:00<00:20, 22.83it/s]

--------------- Epoch 2 ---------------


...

  1%|          | 3/468 [00:00<00:19, 24.37it/s]

--------------- Epoch 122 ---------------


  1%|          | 3/468 [00:00<00:19, 23.36it/s]

--------------- Epoch 123 ---------------


 95%|█████████▍| 444/468 [00:19<00:01, 23.16it/s]